In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(""))))

In [2]:
from src.pipeline import Pipeline

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
pipeline = Pipeline()

In [4]:
pipeline.setup()

In [5]:
pipeline.preprocess()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 449.24it/s]


In [7]:
pipeline.create_all_features()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 243/243 [00:00<00:00, 880.13it/s]


In [8]:
pipeline.features

,qID,pID,y,w2v_cosine,w2v_euclidean,w2v_manhattan,tfidf_cosine,tfidf_euclidean,tfidf_manhattan,glove_cosine,...,polarity_doc,subjectivity_query,polarity_query,bm25,doc_nouns,doc_adjectives,doc_verbs,query_nouns,query_adjectives,query_verbs
0,488467,6256112,1,0.999947,76.877350,629.579689,0.466888,1.032581,3.373521,0.642888,...,0.020000,1.0,0.0,-20.174456,14,7,7,2,1,1
1,124599,4924774,1,0.999595,51.775196,423.578093,0.543206,0.955818,3.091751,-0.294182,...,0.000000,0.0,0.0,-12.668757,7,1,2,2,0,0
2,560377,893530,1,0.999898,46.237732,378.335734,0.257147,1.218895,5.017585,0.576715,...,-0.125000,0.0,0.0,-17.635882,13,2,8,0,2,2
3,1015340,3687433,1,0.999942,25.392345,207.768640,0.341013,1.148030,4.683098,0.560266,...,-0.077778,0.3,0.2,-16.037243,14,4,4,4,1,1
5,932835,3292443,1,0.999736,73.155937,598.398718,0.780465,0.662623,3.123014,0.421301,...,0.116667,0.0,0.0,-29.214098,17,3,9,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,427506,5085340,0,0.999322,70.321327,575.454980,0.373732,1.119168,5.287708,0.479431,...,0.370000,0.0,0.0,-19.536848,19,5,5,3,1,1
134,588340,4322103,0,0.999736,106.265465,869.335519,0.100769,1.341068,6.843310,-0.086400,...,-0.044444,0.0,0.0,-8.574309,26,11,5,1,1,2
135,64644,4141830,0,0.999821,72.929092,596.896216,0.369009,1.123380,4.775432,0.499721,...,0.055556,0.0,0.0,-11.243691,10,3,18,1,0,3
136,55778,1342093,0,0.999753,100.950562,826.062041,0.457584,1.041553,4.544277,0.875458,...,0.200000,0.0,0.0,-9.030987,19,3,8,2,1,0


In [ ]:
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical

logistic_regression_search_space: list = []
logistic_regression_search_space.append(Categorical(['l2', 'none'], name='penalty'))
logistic_regression_search_space.append(Real(0.1, 100.0, name='C'))
logistic_regression_search_space.append(Real(1e-6, 0.1, name='tol'))

In [9]:
pipeline.evaluate('lr', 0, logistic_regression_search_space)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1241.66it/s]


MRR: 0.5119047619047619


/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
import pandas as pd
from src.utils.utils import load

results = load('data/results/results.pkl')
results

,model,hyperparameters,features,sampling,pca,MRR,MAP,nDCG,accuracy,precision,recall,f1,accuracy@50,precision@50,recall@50,f1@50
0,MLPClassifier(),"{""activation"": ""relu"", ""alpha"": 0.0001, ""batch...","[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",243,0,0.511905,1.0,0.726143,0.9958,0.045455,1.0,0.086957,0.58,0.045455,1.0,0.086957
